In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
from shapely.geometry import Point
from matplotlib import pyplot as plt
%matplotlib inline
import time # NB: Use to limit rate of making requests to a website
print('Libraries loaded.')

Libraries loaded.


In [2]:
def get_bounds(x):
    return x.bounds

In [3]:
#shape file
trans_gdf = gpd.read_file('data/ca-transmission-lines/c1ba3265-9d1d-4aa2-88d2-d8e27e2d9e76202048-1-1beav9v.4src.shx')
trans_gdf['length'] = pd.to_numeric(trans_gdf['Length_Fee'])
trans_valid_gdf = trans_gdf[['kV_Sort','length','geometry']].dropna()
print(len(trans_valid_gdf))

6793


In [4]:
MILE_FEET = 5280
trans_long_gdf = trans_valid_gdf[trans_valid_gdf['length'] > MILE_FEET*5]
print(len(trans_long_gdf))

1741


In [7]:
special_gdf = trans_valid_gdf[(trans_valid_gdf['kV_Sort'] > 100) & (trans_valid_gdf['length'] > MILE_FEET*5)].reset_index()
print(len(special_gdf))
special_gdf['bounds'] = pd.DataFrame(special_gdf['geometry'].apply(get_bounds).to_numpy().copy(),columns=['bounds'])
# (minx, miny, maxx, maxy)
max_lat = 34.472499
#restrict to power lines in southern ca
special_gdf = special_gdf[special_gdf['bounds'].apply(lambda x: x[1] < max_lat)]
print(len(special_gdf))

883
235


In [97]:
#check total number of points
def get_npoints(x):
    try:
        return len(x.coords)
    except:
        return sum([len(line.coords) for line in x])
            
n_pts = sum(special_gdf['geometry'].apply(get_npoints))
print(n_pts)

32241


In [55]:
from shapely.geometry.linestring import LineString
from geopy.distance import distance
FAR = (-1000,-100)
def sq_dist(lonlat1,lonlat2): #should use geopy dist here
    return (lonlat1[0]-lonlat2[0])**2 + \
    (lonlat1[1]-lonlat2[1])**2
def find_closest_in_geo(geo,lonlat):
    #closest = FAR
    if type(geo) == LineString:
        best_idx = np.argmin([sq_dist(lonlat,p) for p in geo.coords])
        return geo.coords[int(best_idx)]
    else:
        closest_coords = [
            find_closest_in_geo(line,lonlat) for line in geo
        ]
        best_idx = int(np.argmin([sq_dist(lonlat,p) for p in closest_coords]))
        return closest_coords[best_idx]
def find_closest_trans_point(data,latlon):
    lonlat = latlon[::-1]
    closest_coords = [find_closest_in_geo(special_gdf['geometry'].iloc[i],lonlat)
                     for i in range(len(special_gdf))]
    best_idx = int(np.argmin([sq_dist(lonlat,p) for p in closest_coords]))
    return (best_idx,closest_coords[best_idx][::-1]) #convert back to latlon
latlon = (33.5,-115.5)

%time closest_idx, closest_point = find_closest_trans_point(special_gdf,latlon)
trans_dist = distance(latlon,closest_point).miles
print(closest_point,trans_dist)

CPU times: user 30.5 ms, sys: 2.07 ms, total: 32.6 ms
Wall time: 30.9 ms
(33.3397033880133, -115.558201665591) 11.547963439650962


In [15]:
#special_gdf[['geometry','kV_Sort']].to_file("special_gdf.geojson", driver='GeoJSON')
special_gdf.drop('bounds',axis=1).to_file("special_gdf.geojson", driver='GeoJSON')
with open('special_gdf.geojson','r') as f : 
    special_gdf_geojson = json.load(f)

In [21]:
import gmaps
import os
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])
import json
trans_layer = gmaps.geojson_layer(special_gdf_geojson)

In [54]:
closest_layer = gmaps.symbol_layer(
    [closest_point], fill_color="red", stroke_color="red", scale=2
)
target_layer = gmaps.symbol_layer(
    [latlon], fill_color=(0,255,0), stroke_color=(0,255,0), scale=2
)

fig = gmaps.figure(
    layout={
        'width': '800px',
        'height': '600px',
    },
    map_type='SATELLITE')
fig.add_layer(trans_layer)
fig.add_layer(closest_layer)
fig.add_layer(target_layer)
fig

Figure(layout=FigureLayout(height='600px', width='800px'))

In [147]:
trans_long_gdf['geometry'].iloc[0].coords[2]

(-116.730371024884, 33.7036940161665)